# Лабораторная работа 1. Морфологический анализ

### Задание 1.
1. Изучить демонстрационный ноутбук `"Сегментация и токенизация"`.
1. Выберите наилучшую, на ваш взгляд, библиотеку для сегментации и токенизации.

### Задание 2.
1. Изучите документацию и лицензию морфологического парсера [pymorphy2](https://pymorphy2.readthedocs.io).
1. Установите `pymorphy2`.

### Задание 3.
Напишите функцию `parse_text()`, на вход которых поступает текст (в виде строки), а на выходе формируется список, содержащий для каждого слова входного текста следующую информацию:
- исходную словоформу (`wordform`);
- нормальную форму слова (лемму) (`norm` или `lemma`);
- часть речи (part of speech, `pos`);
- другую грамматическую информацию, выдаваемую `pymorphy2`;
- признак, присутствует ли слово в словаре `pymorphy2`.

Функция должна выбирать наиболее вероятный вариант морфологического разбора слова.


In [28]:
text_example = 'Напишите функцию parse_text(), на вход которых поступает текст (в виде строки), а на выходе формируется список, содержащий для каждого слова входного текста следующую информацию:'

In [29]:
import stanza
import re

In [30]:
stanza.download("ru")

2021-09-20 20:54:45 INFO: Downloading default packages for language: ru (Russian)...


2021-09-20 20:54:46 INFO: File exists: C:\Users\artem\stanza_resources\ru\default.zip.
2021-09-20 20:54:49 INFO: Finished downloading models and saved to C:\Users\artem\stanza_resources.


In [31]:
gr = {
    "NOUN":"имя существительное",
    "ADJF": "имя прилагательное (полное)",
    "NOUN":"имя существительное	хомяк",
    "ADJF":"имя прилагательное (полное)",
    "ADJS":"имя прилагательное (краткое)",
    "COMP":"компаратив",
    "VERB":"глагол (личная форма)",
    "INFN":"глагол (инфинитив)",
    "PRTF":"причастие (полное)",
    "PRTS":"причастие (краткое)",
    "GRND":"деепричастие",
    "NUMR":"числительное",
    "ADVB":"наречие",
    "NPRO":"местоимение-существительное",
    "PRED":"предикатив",
    "PREP":"предлог",
    "CONJ":"союз",
    "PRCL":"частица",
    "INTJ":"междометие",
    "nomn":"именительный",
    "gent":"родительный",
    "datv":"дательный",
    "accs":"винительный",
    "ablt":"творительный",
    "loct":"предложный",
    "voct":"звательный",
    "gen2":"второй родительный (частичный)",
    "acc2":"второй винительный",
    "loc2":"второй предложный (местный)",
    "masc":"мужской род",
    "femn":"женский род",
    "neut":"средний род",
    "LATN":"Токен состоит из латинских букв (например, “foo-bar” или “Maßstab”)",
    "PNCT":"Пунктуация (например, , или !? или …)",
    "NUMB":"Число (например, “204” или “3.14”)",
    "intg":"целое число (например, “204”)",
    "real":"вещественное число (например, “3.14”)",
    "ROMN":"Римское число (например, XI)",
    "UNKN":"Токен не удалось разобрать",
}

In [32]:
ORIGIN_WORD = 'исходаня форма'
NORMAL_FORM = 'нормальная форма'
POS = 'часть речи'
CASE = 'падеж'
GENDER = 'род'
DICT_EXIST = 'наличик в словаре'

In [33]:
import pymorphy2
morph = pymorphy2.MorphAnalyzer()
nlp = stanza.Pipeline(lang='ru', processors='tokenize')

2021-09-20 20:54:50 INFO: Loading these models for language: ru (Russian):
| Processor | Package   |
-------------------------
| tokenize  | syntagrus |

2021-09-20 20:54:50 INFO: Use device: gpu
2021-09-20 20:54:50 INFO: Loading: tokenize
2021-09-20 20:54:50 INFO: Done loading processors!


In [34]:
def get_data(t):
    return 'нет информации' if t is None else t

def get_is_known(t):
    return 'да' if t else 'нет'

def parse_text(text):
    retMap = {}
    t = nlp(text)
    tokens = []
    for sentence in t.sentences:
        tokens += [token.text for token in sentence.tokens]
    for word in tokens:
        if re.match(r'\w+', word):
            parsed = morph.parse(word)[0]
            data = {
                ORIGIN_WORD : word,
                NORMAL_FORM : parsed.normal_form,
                POS         : get_data(gr.get(parsed.tag.POS)),
                CASE        : get_data(gr.get(parsed.tag.case)) ,
                GENDER      : get_data(gr.get(parsed.tag.gender)),
                DICT_EXIST  : get_is_known(morph.word_is_known(word)),
            }
            retMap.update({word : data})              
    return retMap

In [35]:
data = parse_text(text_example)
print(data)

{'Напишите': {'исходаня форма': 'Напишите', 'нормальная форма': 'написать', 'часть речи': 'глагол (личная форма)', 'падеж': 'нет информации', 'род': 'нет информации', 'наличик в словаре': 'да'}, 'функцию': {'исходаня форма': 'функцию', 'нормальная форма': 'функция', 'часть речи': 'имя существительное\tхомяк', 'падеж': 'винительный', 'род': 'женский род', 'наличик в словаре': 'да'}, 'parse_text': {'исходаня форма': 'parse_text', 'нормальная форма': 'parse_text', 'часть речи': 'нет информации', 'падеж': 'нет информации', 'род': 'нет информации', 'наличик в словаре': 'нет'}, 'на': {'исходаня форма': 'на', 'нормальная форма': 'на', 'часть речи': 'предлог', 'падеж': 'нет информации', 'род': 'нет информации', 'наличик в словаре': 'да'}, 'вход': {'исходаня форма': 'вход', 'нормальная форма': 'вход', 'часть речи': 'имя существительное\tхомяк', 'падеж': 'именительный', 'род': 'мужской род', 'наличик в словаре': 'да'}, 'которых': {'исходаня форма': 'которых', 'нормальная форма': 'который', 'част

### Задание 4.
Напишите функцию `save_morph_results()`, сохраняющую структуру данных, получаемую функцией `parse_text()`, в текстовый файл формата JSON.

In [36]:
import json

In [37]:
def save_morph_results(d):
    with open("morph_results.json", "w+", encoding='utf-8') as f:
        tojs = json.dumps(d, indent=4, ensure_ascii=False)
        f.write(tojs)

In [38]:
save_morph_results(parse_text(text_example))

### Задание 5.
Напишите функцию `get_dictionary()`, на вход которой поступает структура данных, получаемая функцией `parse_text()`, а на выходе формируется словарь, ключами которого являются все нормальные формы слов текста, а в качестве значений хранится следующая информация:
- часть речи слова;
- частота слова в тексте;
- все варианты словоформ в тексте с данной нормальной формой.

In [26]:
def get_dictionary(d):
    tmp = {}
    
    return tmp

In [27]:
dt = get_dictionary(parse_text(text_example))
print(dt)

{}


### Задание 6.
Напишите функцию `save_dictionary()`, сохраняющую предыдущую структуру данных в текстовый файл формата JSON. Слова в файле должны быть упорядочены по убыванию частоты.

In [5]:
def save_dictionary():
    # Ваш код здесь
    return

### Задание 7.
Напишите функцию `get_non_dict()`, на вход которой поступает структура данных, получаемая функцией `parse_text()`, а на выходе формируется словарь, содержащий слова текста, отсутствующие в словаре `pymorphy2`, вместе с частотой слова в тексте.

In [6]:
def get_non_dict():
    # Ваш код здесь
    return

### Задание 8.
Напишите функцию `save_non_dict()`, сохраняющую структуру данных, получаемую функцией `get_non_dict()`, в текстовый файл формата TSV (tab-separated values). Слова в файле должны быть упорядочены по убыванию частоты.

In [7]:
def save_non_dict():
    # Ваш код здесь
    return

### Задание 9.
Напишите функцию `get_pos_distribution()`, на вход которой поступает словарь, формируемый функцией `get_dictionary()`, а на выходе выдается структура данных, содержащая частотное распределение частей речи в словаре:
- часть речи – количество уникальных слов – общее количество слов.

In [8]:
def get_pos_distribution():
    # Ваш код здесь
    return

### Задание 10.
Создайте текстовую коллекцию - скачайте не менее **10 текстов** разных жанров и разного размера (например, произведения классиков, современных писателей, новостные статьи, научные статьи и т.п.).

Учитывайте кодировку – все файлы должны быть в UTF-8.

In [9]:
# Ваш код для открытия файлов.

### Задание 11.
Обработайте текстовую коллекцию при помощи функций `parse_text()`, `get_dictionary()` и `get_non_dict()`, и сохраните результаты в текстовых файлах при помощи функций `save_morph_results()`, `save_dictionary()` и `save_non_dict()`.

Для каждого входного текстового файла выведите на экран следующую информацию:
- имя файла;
- размер файла в байтах;
- размер текста в словах (общее количество слов в тексте);
- размер словаря (количество уникальных слов в тексте);
- время обработки файла (в секундах).

In [10]:
# Ваш код для обработки коллекции и вывода результатов

### Задание 12.
Для самого большого словаря выполните следующее:
- постройте частотное распределение слов: по оси ординат – частота, по оси абсцисс – слова, упорядоченные по убыванию частоты (по-другому, *ранги* слов). В качестве значений на оси абсцисс можно выводить ранги слов, а не сами слова;
- постройте в виде диаграммы распределение частей речи, полученное функцией `get_pos_distribution()`.

In [11]:
# Ваш код для вывода графика и диаграммы

### Задание 13.
Постройте график зависимость времени морфологического анализа от размера текстового файла.

In [12]:
# Ваш код для вывода графика